In [1]:
import numpy as np
import pandas as pd

In [ ]:
层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（即两个以上）索引级别。
抽象来说，它使你能以低纬度形式处理高维度数据。

In [3]:
# 看一个简单的例子：创建一个Series，并用一个由列表或数组组成的列表作为索引:
data = pd.Series(np.random.randn(9), 
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], 
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.127552
   2    1.062935
   3    0.677409
b  1    0.700570
   3   -1.592250
c  1    0.324981
   2   -0.090097
d  2   -1.909799
   3   -0.135144
dtype: float64

In [ ]:
看到的结果是经过美化的，带有MultiIndex索引的Series的格式。

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [ ]:
如何理解将MultiIndex分为"levels"与"labels"两部分?
这种数据结构会为表示高维度数据带来怎样的便利?

In [7]:
# 对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单：
data['b']

1    0.70057
3   -1.59225
dtype: float64

In [10]:
data['b':'c']

b  1    0.700570
   3   -1.592250
c  1    0.324981
   2   -0.090097
dtype: float64

In [11]:
data.loc[['b', 'd']]

b  1    0.700570
   3   -1.592250
d  2   -1.909799
   3   -0.135144
dtype: float64

In [12]:
# 有时还可以在“内层中”进行选取：
data.loc[:, 2] # 跳过outest level，对次外层中元素进行选取

a    1.062935
c   -0.090097
d   -1.909799
dtype: float64

In [ ]:
层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色

In [14]:
# 例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中：
data.unstack()

,1,2,3
a,0.127552,1.062935,0.677409
b,0.700570,NaN,-1.592250
c,0.324981,-0.090097,NaN
d,NaN,-1.909799,-0.135144


In [ ]:
本例中，data的MultiIndex是一个二维数组，即可以通过一个二维向量来确定data中每个元素的label，
故当然可以将data转化为一个DataFrame。
需要注意的是，当处理高维数据时，unstack()能否将"Series with MultiIndex"转换为DataFrame?

In [ ]:
对于一个DataFrame，每条轴都可以有分层索引：

In [18]:
frame1 = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
frame1

0   1   2
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

In [22]:
frame2 = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                      index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                      columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])
frame2

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [ ]:
index（行索引）操作行元素，columns（列索引）操作列元素。
length of MultiIndex需要与length of axe长度相同，在本例中，index每一维长度均为4，columns每一维长度均为3
需要注意，这里的"长度"为实际的操作长度，因为有的函数会自动处理column与row的长度

In [26]:
# 各层都可以有名字（可以是字符串，也可以是别的python对象）。
# 如果指定了名称，它们就会显示在控制台输出中：
frame2.index.names = ['key1', 'key2']
frame2.columns.names = ['state', 'color']
frame2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [27]:
# 有了部分列索引，因此可以轻松选取列分组：
frame2['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [ ]:
@重排与分级排序

In [ ]:
有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。

In [ ]:
# swaplevel接受两个level(级别)编号或名称，并返回一个互换了level的新对象（但数据不会发生变化）：

In [29]:
frame2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [30]:
frame2.swaplevel('key1', 'key2') # 调整level，value不变

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [35]:
# 而sort_index则根据"行索引"上的"单个级别"中的值对数据进行排序：
frame2.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [32]:
# 交换level时，常常组合使用swaplevel()与sort_index()：
frame2.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [ ]:
@根据级别（level）汇总统计

In [ ]:
许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于制定在某条轴上求和的级别。

In [40]:
frame2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [36]:
# 再以frame2为例，我们可以根据行或者列上的级别来进行求和：
frame2.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [38]:
frame2.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [ ]:
这其实是利用了pandas的groupby功能，本书稍后将对其进行详细讲解。

In [ ]:
@使用DataFrame的列进行索引

In [ ]:
人们经常想要将DataFrame的一个或多个column当作index(行索引)来使用,或者可能希望将行索引变成DataFrame的列。

In [42]:
# 以下面这个DataFrame为例：
frame3 = pd.DataFrame({'a':range(7), 
                       'b':range(7, 0, -1),
                       'c':['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                       'd':[0, 1, 2, 0, 1, 2, 3]})
frame3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [43]:
# DataFrame的set_index函数会将df其中的一个或多个column转换为index，并创建一个新的DataFrame：
frame4 = frame3.set_index(['c', 'd'])
frame4

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [44]:
# 默认情况下，转换后的column会在原df中被移除，但也可以将其保留下来：
frame3.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [45]:
# reset_index的功能刚好与set_index相反，层次化索引的级别会被转移到列中：
frame4.reset_index() # 重置了所有的层次化索引

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [ ]:
author: @ melo
last updated: 2018-10-15